## Segmenting and Clustering Neighborhoods in Delhi

## Introduction

In this lab,we will do web scrapping of wikepedia Page representing Districts of Delhi. First we install some important libaries then do web scrapping through Beautiful Soup.Then we clean the data by removing non assigned values,unncessary characters and lastly we compare our DataFrame with Dataframe in the ask assignment.After that we will use Geosptial Data and merge it to our previous DataFrame.Get venues of Districts of Delhi by using foruquare api and then add clusters to to get the information in clustered form.

## Table of Contents

1. Import the Libraries
2. Scrap the Data through Beautifulsoup4
3. Clean the Data
4. Open and prepare Geospatial Coordinate file 
5. Visualization of Districts of Delhi
6. Use Forsuare api to get nearby venues
7. Analyze the Delhi Districts
8. Use K-means cluster 
9. Examine the clusters

## 1.Import the Libraries


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy #install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# To praise HTML and XML documents
!pip install beautifulsoup4
from bs4 import BeautifulSoup

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')


ERROR: Invalid requirement: '#install'


Libraries imported.


In [2]:
## 2. Scrap the Data through Beautifulsoup4

In [3]:
# Specify the url of the page we want to scrap
url='https://simple.wikipedia.org/wiki/List_of_districts_in_Delhi'

#Send HTTP request to specify the url and then stor it into Data.
Data = requests.get(url)

# BeautifulSoup library  is built on the top of the HTML parsing libraries like html5lib, lxml, html.parser, etc. 
# For simplicity we are using html5lib it specifies the HTML parser.
soup=BeautifulSoup(Data.content,'html.parser')

#soup.prettify() gives the visual representation of the parse tree created from the raw HTML content.
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of districts in Delhi - Simple English Wikipedia, the free encyclopedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"31445e7a-b6f7-453d-8478-72b8700a584f","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_districts_in_Delhi","wgTitle":"List of districts in Delhi","wgCurRevisionId":6890806,"wgRevisionId":6890806,"wgArticleId":598674,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with citations lacking titles","CS1 errors: external links","Pages using 

In [4]:
## 3. Clean the Data

In [5]:
# Create List to store Data
District=[]
Headquarters=[]
#Use for loop to find Data we need
#Append call Data in respectively in above three lists
for row in soup.find('table').find_all('tr'):
    Code=row.find_all('td')
    if(len(Code)>0):
        District.append(Code[1].text.rstrip('\n'))
        Headquarters.append(Code[2].text.rstrip('\n'))
        


#  Convert List into new DataFrame     
Delhi_Districts=pd.DataFrame({'District':District,
                             'Headquarters':Headquarters,
                             
                             })


Delhi_Districts


,District,Headquarters
0,New Delhi,Connaught Place
1,North West Delhi,Narela
2,North Delhi,Kanjhawala
3,West Delhi,Rajouri Garden
4,South West Delhi,Dwarka
5,South Delhi,Saket
6,South East Delhi,Defence Colony
7,Central Delhi,Daryaganj
8,North East Delhi,Seelampur
9,Shahdara,Shahdara


## 4. Open and prepare Geospatial Coordinate file 

In [6]:
# To find Latitude and Longitude of Delhi
home='Delhi'
geo=Nominatim(user_agent='my-application')
loc=geo.geocode(home)
latitude=loc.latitude
longitude=loc.longitude
print('Latitude and longitude of Delhi are {},{} respectively.'.format(latitude,longitude))


Latitude and longitude of Delhi are 28.6517178,77.2219388 respectively.


In [7]:
# We have already prepare the file of Geospatial Coordiantes in Excel
Geo_delhi_data=pd.read_excel('Geo_delhi.xlsx')
Geo_delhi_data.dropna(axis='columns',inplace=True)
Coord =Geo_delhi_data

In [8]:
Delhi_neighbor=Delhi_Districts.merge(Coord,on='District',how='left')
Delhi_neighbor

,District,Headquarters,Latitude,Longitude
0,New Delhi,Connaught Place,28.614179,77.202266
1,North West Delhi,Narela,28.614179,77.202266
2,North Delhi,Kanjhawala,28.614179,77.202266
3,West Delhi,Rajouri Garden,28.647952,77.085565
4,South West Delhi,Dwarka,28.586448,76.979153
5,South Delhi,Saket,28.485169,77.196380
6,South East Delhi,Defence Colony,28.544441,77.272873
7,Central Delhi,Daryaganj,28.698548,77.219391
8,North East Delhi,Seelampur,28.723308,77.266857
9,Shahdara,Shahdara,28.673333,77.289025


## 5. Visualization of Districts of Delhi

In [9]:
# Use folium to represent Districts in the map of Delhi
Delhi_map = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat,lng, district, headquarters in zip(
                                            Delhi_neighbor['Latitude'],
                                            Delhi_neighbor['Longitude'],
                                            Delhi_neighbor['District'],
                                            Delhi_neighbor['Headquarters']):
    label = '{}, {}'.format(district, headquarters)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Delhi_map)  
    
Delhi_map



## 6. Use Forsuare api to get nearby venues

In [10]:
CLIENT_ID = 'OI1I4K015NRA5LJSTGYRCJMEC5ODYZOC4AXK4WBWC34JB3SA' # your Foursquare ID
CLIENT_SECRET = 'DR112UARCRCCPJQSCQ5JT4WI1F0FXKJTTVBCXOGVZR1JWRLL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT=100

Your credentails:
CLIENT_ID: OI1I4K015NRA5LJSTGYRCJMEC5ODYZOC4AXK4WBWC34JB3SA
CLIENT_SECRET:DR112UARCRCCPJQSCQ5JT4WI1F0FXKJTTVBCXOGVZR1JWRLL


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
Delhi_neighbor = getNearbyVenues(names=Delhi_neighbor['District'],
                                   latitudes=Delhi_neighbor['Latitude'],
                                   longitudes=Delhi_neighbor['Longitude']
                                  ) 


New Delhi
North West Delhi
North Delhi
West Delhi
South West Delhi
South Delhi
South East Delhi
Central Delhi
North East Delhi
Shahdara
East Delhi


In [12]:
print('There are {} uniques categories.'.format(len(Delhi_neighbor['Venue Category'].unique())))

There are 12 uniques categories.


## 7. Analyze the Delhi Districts

In [13]:
# one hot encoding
Delhi_onehot = pd.get_dummies(Delhi_neighbor[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Delhi_onehot['District'] = Delhi_neighbor['District'] 
# move neighborhood column to the first column
fixed_columns =list(Delhi_onehot.columns[-1:]) + list(Delhi_onehot.columns[:-1])
Delhi_onehot = Delhi_onehot[fixed_columns]

Delhi_onehot

,District,ATM,Auto Workshop,Bakery,Cosmetics Shop,Garden,Gift Shop,Gym,Light Rail Station,Museum,Park,Platform,Vegetarian / Vegan Restaurant
0,New Delhi,0,0,0,0,1,0,0,0,0,0,0,0
1,New Delhi,0,0,0,0,0,0,0,0,1,0,0,0
2,North West Delhi,0,0,0,0,1,0,0,0,0,0,0,0
3,North West Delhi,0,0,0,0,0,0,0,0,1,0,0,0
4,North Delhi,0,0,0,0,1,0,0,0,0,0,0,0
5,North Delhi,0,0,0,0,0,0,0,0,1,0,0,0
6,West Delhi,0,0,1,0,0,0,0,0,0,0,0,0
7,West Delhi,1,0,0,0,0,0,0,0,0,0,0,0
8,South East Delhi,0,0,0,0,0,0,0,1,0,0,0,0
9,South East Delhi,0,1,0,0,0,0,0,0,0,0,0,0


In [14]:
# Reset index
Delhi_onehot = Delhi_onehot.groupby('District').mean().reset_index()
Delhi_onehot

,District,ATM,Auto Workshop,Bakery,Cosmetics Shop,Garden,Gift Shop,Gym,Light Rail Station,Museum,Park,Platform,Vegetarian / Vegan Restaurant
0,Central Delhi,0.0,0.0,0.0,0.5,0.0,0.00,0.0,0.00,0.0,0.5,0.00,0.00
1,East Delhi,0.0,0.0,0.0,0.0,0.0,0.00,1.0,0.00,0.0,0.0,0.00,0.00
2,New Delhi,0.0,0.0,0.0,0.0,0.5,0.00,0.0,0.00,0.5,0.0,0.00,0.00
3,North Delhi,0.0,0.0,0.0,0.0,0.5,0.00,0.0,0.00,0.5,0.0,0.00,0.00
4,North East Delhi,1.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00
5,North West Delhi,0.0,0.0,0.0,0.0,0.5,0.00,0.0,0.00,0.5,0.0,0.00,0.00
6,Shahdara,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.25,0.25
7,South East Delhi,0.0,0.5,0.0,0.0,0.0,0.00,0.0,0.50,0.0,0.0,0.00,0.00
8,West Delhi,0.5,0.0,0.5,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00


In [15]:
# Show top 5 Venues in each District
num_top_venues = 5

for hood in Delhi_onehot['District']:
    print("----"+hood+"----")
    temp = Delhi_onehot[Delhi_onehot['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Delhi----
            venue  freq
0  Cosmetics Shop   0.5
1            Park   0.5
2             ATM   0.0
3   Auto Workshop   0.0
4          Bakery   0.0


----East Delhi----
            venue  freq
0             Gym   1.0
1             ATM   0.0
2   Auto Workshop   0.0
3          Bakery   0.0
4  Cosmetics Shop   0.0


----New Delhi----
           venue  freq
0         Garden   0.5
1         Museum   0.5
2            ATM   0.0
3  Auto Workshop   0.0
4         Bakery   0.0


----North Delhi----
           venue  freq
0         Garden   0.5
1         Museum   0.5
2            ATM   0.0
3  Auto Workshop   0.0
4         Bakery   0.0


----North East Delhi----
            venue  freq
0             ATM   1.0
1   Auto Workshop   0.0
2          Bakery   0.0
3  Cosmetics Shop   0.0
4          Garden   0.0


----North West Delhi----
           venue  freq
0         Garden   0.5
1         Museum   0.5
2            ATM   0.0
3  Auto Workshop   0.0
4         Bakery   0.0


----Shahdara-

In [16]:
# Define function to return the common Venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
# Represent  Top 10 Venues in each Districts
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['District'] = Delhi_onehot['District']

for ind in np.arange(Delhi_onehot.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Delhi_onehot.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Delhi,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum,Light Rail Station,Gym,Gift Shop,Garden,Bakery
1,East Delhi,Gym,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gift Shop,Garden,Cosmetics Shop,Bakery
2,New Delhi,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park,Light Rail Station,Gym,Gift Shop,Cosmetics Shop,Bakery
3,North Delhi,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park,Light Rail Station,Gym,Gift Shop,Cosmetics Shop,Bakery
4,North East Delhi,ATM,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gym,Gift Shop,Garden,Cosmetics Shop
5,North West Delhi,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park,Light Rail Station,Gym,Gift Shop,Cosmetics Shop,Bakery
6,Shahdara,Vegetarian / Vegan Restaurant,Platform,Light Rail Station,Gift Shop,Park,Museum,Gym,Garden,Cosmetics Shop,Bakery
7,South East Delhi,Light Rail Station,Auto Workshop,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Gym,Gift Shop,Garden,Cosmetics Shop
8,West Delhi,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gym,Gift Shop,Garden


## 8. Use K-means cluster 

In [18]:
# set number of clusters
kclusters = 4
Delhi_neighbor_clustering = Delhi_onehot.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Delhi_neighbor_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 0, 0, 3, 0, 1, 1, 3])

In [19]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Delhi_merged=Delhi_neighbor
Delhi_merged =Delhi_merged.join(neighborhoods_venues_sorted.set_index('District'), on='District',how='right')

Delhi_merged # check the last columns!

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New Delhi,28.614179,77.202266,Mughal Gardens | मुगल गार्डन (Mughal Gardens),28.617427,77.199727,Garden,0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park,Light Rail Station,Gym,Gift Shop,Cosmetics Shop,Bakery
1,New Delhi,28.614179,77.202266,"Rashtrapati Bhawan, President's Estate Yeni De...",28.614350,77.199715,Museum,0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park,Light Rail Station,Gym,Gift Shop,Cosmetics Shop,Bakery
2,North West Delhi,28.614179,77.202266,Mughal Gardens | मुगल गार्डन (Mughal Gardens),28.617427,77.199727,Garden,0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park,Light Rail Station,Gym,Gift Shop,Cosmetics Shop,Bakery
3,North West Delhi,28.614179,77.202266,"Rashtrapati Bhawan, President's Estate Yeni De...",28.614350,77.199715,Museum,0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park,Light Rail Station,Gym,Gift Shop,Cosmetics Shop,Bakery
4,North Delhi,28.614179,77.202266,Mughal Gardens | मुगल गार्डन (Mughal Gardens),28.617427,77.199727,Garden,0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park,Light Rail Station,Gym,Gift Shop,Cosmetics Shop,Bakery
5,North Delhi,28.614179,77.202266,"Rashtrapati Bhawan, President's Estate Yeni De...",28.614350,77.199715,Museum,0,Museum,Garden,Vegetarian / Vegan Restaurant,Platform,Park,Light Rail Station,Gym,Gift Shop,Cosmetics Shop,Bakery
6,West Delhi,28.647952,77.085565,Cup Cake Radisson Blu,28.646223,77.086960,Bakery,3,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gym,Gift Shop,Garden
7,West Delhi,28.647952,77.085565,Axis Bank ATM,28.650040,77.087040,ATM,3,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gym,Gift Shop,Garden
8,South East Delhi,28.544441,77.272873,Okhla Metro Station,28.543125,77.275562,Light Rail Station,1,Light Rail Station,Auto Workshop,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Gym,Gift Shop,Garden,Cosmetics Shop
9,South East Delhi,28.544441,77.272873,Kashyap Fiat,28.548672,77.271178,Auto Workshop,1,Light Rail Station,Auto Workshop,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Gym,Gift Shop,Garden,Cosmetics Shop


In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Delhi_merged['Venue Latitude'],
                                  Delhi_merged['Venue Longitude'], 
                                  Delhi_merged['District'], 
                                  Delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 9. Examine the clusters

In [21]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 1, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,District Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,28.544441,77.275562,Light Rail Station,1,Light Rail Station,Auto Workshop,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Gym,Gift Shop,Garden,Cosmetics Shop
9,28.544441,77.271178,Auto Workshop,1,Light Rail Station,Auto Workshop,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Gym,Gift Shop,Garden,Cosmetics Shop
10,28.698548,77.220394,Park,1,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum,Light Rail Station,Gym,Gift Shop,Garden,Bakery
11,28.698548,77.216670,Cosmetics Shop,1,Park,Cosmetics Shop,Vegetarian / Vegan Restaurant,Platform,Museum,Light Rail Station,Gym,Gift Shop,Garden,Bakery
15,28.673333,77.289011,Light Rail Station,1,Vegetarian / Vegan Restaurant,Platform,Light Rail Station,Gift Shop,Park,Museum,Gym,Garden,Cosmetics Shop,Bakery
16,28.673333,77.288999,Gift Shop,1,Vegetarian / Vegan Restaurant,Platform,Light Rail Station,Gift Shop,Park,Museum,Gym,Garden,Cosmetics Shop,Bakery
17,28.673333,77.290126,Vegetarian / Vegan Restaurant,1,Vegetarian / Vegan Restaurant,Platform,Light Rail Station,Gift Shop,Park,Museum,Gym,Garden,Cosmetics Shop,Bakery
18,28.673333,77.292520,Platform,1,Vegetarian / Vegan Restaurant,Platform,Light Rail Station,Gift Shop,Park,Museum,Gym,Garden,Cosmetics Shop,Bakery


In [22]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 2, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,District Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,28.620477,77.306951,Gym,2,Gym,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gift Shop,Garden,Cosmetics Shop,Bakery


In [23]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 3, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,District Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,28.647952,77.08696,Bakery,3,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gym,Gift Shop,Garden
7,28.647952,77.08704,ATM,3,Bakery,ATM,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gym,Gift Shop,Garden
12,28.723308,77.26387,ATM,3,ATM,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gym,Gift Shop,Garden,Cosmetics Shop
13,28.723308,77.26387,ATM,3,ATM,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gym,Gift Shop,Garden,Cosmetics Shop
14,28.723308,77.26780,ATM,3,ATM,Vegetarian / Vegan Restaurant,Platform,Park,Museum,Light Rail Station,Gym,Gift Shop,Garden,Cosmetics Shop


In [24]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 4, Delhi_merged.columns[[1] + list(range(5, Delhi_merged.shape[1]))]]

,District Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
